## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import tabula

## Read-in

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.io.read_pdf('condo.pdf', pages='all',lattice=True)

## Clean Data

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## Geocode

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [6]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [7]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction section

In [8]:
df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,geocoded,lat,lon
1,Oceanside @ Fisher Island\r7954 Fisher Island ...,3/15/2023,346,"$21,000,000.00",6822,"$3,078.28",Dora Puig,Luxe Living Realty,Julian Johnston,The Corcoran Group,Oceanside @ Fisher Island 7954 Fisher Island D...,Oceanside @ Fisher Island,7954 Fisher Island Dr 7954 Miami Beach,"(25.7589149, -80.13751099999999)",25.758915,-80.137511
2,Ocean Park South Beach\r312 Ocean Dr PH2\rMiam...,3/13/2023,229,"$6,450,000.00",4662,"$2,695.36",Eloy Carmenate,The Corcoran Group,Eloy Carmenate,The Corcoran Group,Ocean Park South Beach 312 Ocean Dr PH2 Miami ...,Ocean Park South Beach,312 Ocean Dr PH2 Miami Beach,"(25.7722279, -80.1325823)",25.772228,-80.132582
3,Fisher Island\r7912 Fisher Island Dr 7912\rMia...,3/17/2023,18,"$5,750,000.00",2530,"$2,272.73",Nataly Stuart,"Socialite Realty, LLC",Nataly Stuart,"Socialite Realty, LLC",Fisher Island 7912 Fisher Island Dr 7912 Miami...,Fisher Island,7912 Fisher Island Dr 7912 Miami Beach,"(25.7589149, -80.13751099999999)",25.758915,-80.137511
4,Bayview\r5325 Fisher Island Dr 5325\rMiami Beach,3/15/2023,114,"$3,900,000.00",2740,"$1,423.36",Jonathan Kotler,Keller Williams Elite Properties,Ana Rodrigues Da Cruz,Coldwell Banker Realty,Bayview 5325 Fisher Island Dr 5325 Miami Beach,Bayview,5325 Fisher Island Dr 5325 Miami Beach,"(25.763618, -80.148631)",25.763618,-80.148631
5,Solimar Condo\r9595 Collins Ave N5-H & N5-I\rS...,3/15/2023,25,"$3,415,000.00",3730,$915.55,Allen Davoudpour,"Compass Florida, LLC",Shneur Shapira,"Brokerage S Florida Real Estate, LLC.",Solimar Condo 9595 Collins Ave N5-H & N5-I Sur...,Solimar Condo,9595 Collins Ave N5-H & N5-I Surfside,"(25.8865012, -80.12202219999999)",25.886501,-80.122022
6,Acqualina\r17875 Collins Ave 1705\rSunny Isles...,3/13/2023,146,"$2,550,000.00",2214,"$1,151.76",Anna Burchill,BrokerNation Real Estate,Romina Iseruk,Lumer Real Estate,Acqualina 17875 Collins Ave 1705 Sunny Isles B...,Acqualina,17875 Collins Ave 1705 Sunny Isles Beach,"(25.9408209, -80.1209181)",25.940821,-80.120918
7,Brickell Heights\r55 SW 9th St PH4606\rMiami,3/14/2023,30,"$2,550,000.00",2837,"$1,077.77",Jonathan Mann,Coldwell Banker Realty,Luis Gautier,Fortune Christie's Int'l Real Estate,Brickell Heights 55 SW 9th St PH4606 Miami,Brickell Heights,55 SW 9th St PH4606 Miami,"(25.7656252, -80.1938705)",25.765625,-80.193871
8,W South Beach\r2201 Collins Ave 1014\rMiami Beach,3/13/2023,16,"$2,400,000.00",1346,"$2,085.14",Christopher Adeleke,Douglas Elliman,Taylor Travaglione,One Sotheby's International Realty,W South Beach 2201 Collins Ave 1014 Miami Beach,W South Beach,2201 Collins Ave 1014 Miami Beach,"(25.7973813, -80.1271903)",25.797381,-80.127190
9,Grand Bay Tower Condo\r430 Grand Bay Dr 107\rK...,3/15/2023,18,"$2,220,000.00",2470,$898.79,Carlos Castro,NestWise Real Estate LLC,Angelica Gomez,"Related ISG Realty, LLC.",Grand Bay Tower Condo 430 Grand Bay Dr 107 Key...,Grand Bay Tower Condo,430 Grand Bay Dr 107 Key Biscayne,"(25.693091, -80.157996)",25.693091,-80.157996
10,0Solimar Condo\r9595 Collins Ave N5-H\rSurfside,3/15/2023,25,"$2,000,000.00",1980,"$1,010.10",Allen Davoudpour,"Compass Florida, LLC",Shneur Shapira,"Brokerage S Florida Real Estate, LLC.",0Solimar Condo 9595 Collins Ave N5-H Surfside,,0Solimar Condo 9595 Collins Ave N5-H Surfside,"(25.8867975, -80.1221833)",25.886798,-80.122183


In [9]:
df.at[10,'building_name']=('Solimar Condo')
df.at[10,'address_only']=('9595 Collins Avenue')

df.at[2,'Agent']=('Mick Duchon & Eloy Carmenate')
df.at[2,'Buyer Agent']=('Mick Duchon & Eloy Carmenate')

## Format Data

In [10]:
### Insert NaNs if needed ###
df = df.replace('N/A', np.nan)

In [11]:
df['int_Sale_Price'] = df['Sale Price'].str.replace('$','',regex=True)
df['int_Sale_Price'] = df['int_Sale_Price'].str.replace(',','',regex=True)
df['int_Sale_Price'] = pd.to_numeric(df['int_Sale_Price'])

df['price_per_sqft'] = df['price_per_sqft'].str.replace('$','',regex=True)
df['price_per_sqft'] = df['price_per_sqft'].str.replace(',','',regex=True)
df['price_per_sqft'] = pd.to_numeric(df['price_per_sqft'])

df['days_on_market'] = pd.to_numeric(df['days_on_market'])

## Color-code top sale

In [12]:
### Insert RANK values ###
df['RANK'] = range(1, len(df) + 1)
# use numpy to assign values to the 'COLOR' column
df['COLOR'] = np.where(df['RANK'] <= 1, 'orange', 'blue')

In [13]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only',
       'geocoded', 'lat', 'lon', 'int_Sale_Price', 'RANK', 'COLOR'],
      dtype='object')

## HTML Popup Formatter

In [14]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only',
       'geocoded', 'lat', 'lon', 'int_Sale_Price', 'RANK', 'COLOR'],
      dtype='object')

In [15]:
def popup_html(row):
    building_name = row['building_name']
    price = row['Sale Price']
    days_on_market = row['days_on_market']
    listing_agent = row['Agent']
    buyers_agent = row['Buyer Agent']
    psf = row['Sq. Ft.']
    address = row['address_only']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>{}'''.format(price) + '''<br>
    <strong>Days on Market: </strong>{}'''.format(days_on_market) + '''<br>
    <strong>Listing Agent: </strong>{}'''.format(listing_agent) + '''<br>
    <strong>Buyer's Agent: </strong>{}'''.format(buyers_agent) + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(psf) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    </html>
    '''
    return html

In [16]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('March 12th - March 18th')


### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [17]:
m.save('index.html')

## Data snagger

In [18]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [19]:
### Highest and lowest sale price ###
print(f"{ME}{BR}{df.loc[df['int_Sale_Price'].idxmax()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmax()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmax()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmax()]['days_on_market']}")
print(f"{LE}{BR}{df.loc[df['int_Sale_Price'].idxmin()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmin()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmin()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmin()]['days_on_market']}")
### Highest and lowest psf ###
print(f"{MAX_PSF}{BR}{df.loc[df['price_per_sqft'].idxmax()]['building_name']}, {df.loc[df['price_per_sqft'].idxmax()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmax()]['Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmax()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmax()]['days_on_market']}")
print(f"{MIN_PSF}{BR}{df.loc[df['price_per_sqft'].idxmin()]['building_name']}, {df.loc[df['price_per_sqft'].idxmin()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmin()]['Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmin()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmin()]['days_on_market']}")
### Highest and lowest days on market ###
print(f"{DAYS_MAX}{BR}{df.loc[df['days_on_market'].idxmax()]['building_name']}, {df.loc[df['days_on_market'].idxmax()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmax()]['Agent']} with {df.loc[df['days_on_market'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmax()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmax()]['days_on_market']}")
print(f"{DAYS_MIN}{BR}{df.loc[df['days_on_market'].idxmin()]['building_name']}, {df.loc[df['days_on_market'].idxmin()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmin()]['Agent']} with {df.loc[df['days_on_market'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmin()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmin()]['days_on_market']}")

Most Expensive
Oceanside @ Fisher Island , 7954 Fisher Island Dr 7954 Miami Beach | Price $21,000,000 | $3,078 psf | Listing agent: Dora Puig with Luxe Living Realty | Buyer's agent: Julian Johnston with The Corcoran Group | Days on market: 346
Least Expensive
Solimar Condo, 9595 Collins Avenue | Price $2,000,000 | $1,010 psf | Listing agent: Allen Davoudpour with Compass Florida, LLC | Buyer's agent: Shneur Shapira with Brokerage S Florida Real Estate, LLC. | Days on market: 25
Highest Price Per Square Foot
Oceanside @ Fisher Island , 7954 Fisher Island Dr 7954 Miami Beach | Price $21,000,000 | $3,078 psf | Listing agent: Dora Puig with Luxe Living Realty | Buyer's agent: Julian Johnston with The Corcoran Group | Days on market: 346
Lowest Price Per Square Foot
Grand Bay Tower Condo , 430 Grand Bay Dr 107 Key Biscayne | Price $2,220,000 | $899 psf | Listing agent: Carlos Castro with NestWise Real Estate LLC | Buyer's agent: Angelica Gomez with Related ISG Realty, LLC. | Days on market

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [20]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [21]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_03202023
